# **PROGRAM MENGHITUNG KATA**

**Dibuat oleh :**

Muhammad Idham Ma'arif

245150300111024

Teknik Komputer 2024

---

<br>

# 1. Import library yang akan dibutuhkan

- **threading** : Untuk multithreading
- **re**        : Pencarian kata dengan regex
- **pathlib.Path** : Mengakses file
- **pandas** : output terstruktur (DataFrame)

In [1]:
import threading
import re
from pathlib import Path
import pandas as pd

<br>

# 2. Deklarasi global variabel

- **WORD_RE** digunakan untuk pencarian kata dengan regex
- **jumlahKata** untuk menyimpan nama *kata* dan jumlah *kata* tersebut muncul
- **lock** untuk menggabungkan penemuan lokal dengan variabel global

In [16]:
WORD_RE = re.compile(r"[A-Za-z']+")
jumlahKata = {}
lock = threading.Lock()

<br>

# 3. Buat fungsi untuk menghitung kata

Fungsi **`HitungKataDalamFile(files)`** digunakan untuk menghitung frekuensi kemunculan setiap kata dalam satu atau beberapa file teks.

1. Inisialisasi lokal dengan membuat dictionary lokal untuk menyimpan jumlah kata sementara dari file yang sedang diproses.

2. Baca dan normalisasi teks dengan cara setiap file dalam files dibaca menggunakan **`Path(fname).read_text()`**. Teks dikonversi ke huruf kecil `(lower())` agar pencatatan kata tidak peka huruf besar/kecil.

3. Ekstraksi kata dengan regex dengan cara Kata-kata diambil menggunakan pola **`WORD_RE.findall(text)`**.

4. Setiap kata ditambahkan ke lokal dengan menghitung jumlah kemunculannya.

5. Sinkronisasi hasil (thread-safe). Hasil lokal digabungkan ke dictionary global jumlahKata.

Proses ini dilakukan dalam blok with lock: untuk menghindari race condition saat dijalankan oleh banyak thread secara paralel.

In [17]:
# menghitung jumlah kata dalam file (fungsi)
def HitungKataDalamFile(files):
    lokal = {}
    for fname in files:
        text = Path(fname).read_text(encoding="utf-8", errors="ignore").lower()
        for w in WORD_RE.findall(text):
            lokal[w] = lokal.get(w, 0) + 1
    # gabungkan ke global variabel dengan lock
    with lock:
        for w, c in lokal.items():
            jumlahKata[w] = jumlahKata.get(w, 0) + c

<br>

# 4. Buat fungsi main untuk menggunakan fungsi diatas

Deskripsi Kode main()

1. Daftar file contoh. Variabel files berisi daftar nama file teks `(a.txt, b.txt, c.txt)`yang akan dianalisis.

2. Pembagian kerja ke thread yaitu file dibagi dua bagian menggunakan **`mid = len(files)//2`**. Dua thread (t1 dan t2) dibuat untuk memanggil fungsi `HitungKataDalamFile` secara paralel:

    - t1 memproses bagian pertama files[:mid].

    - t2 memproses bagian kedua files[mid:].

3. Menjalankan thread

    - t1.start() dan t2.start() digunakan untuk memulai eksekusi thread.

    - t1.join() dan t2.join() memastikan program menunggu kedua thread selesai sebelum melanjutkan.

4. Membuat `DataFrame` hasil: Setelah semua kata dihitung dan disimpan dalam dictionary global jumlahKata, hasilnya diubah menjadi DataFrame pandas.

5. Data diurutkan (sorted) berdasarkan frekuensi kata, dari yang paling sering muncul ke yang paling jarang (reverse=True).

6. Menampilkan hasil. **`print(df.to_string())`** menampilkan DataFrame lengkap ke layar tanpa pemotongan baris.

In [24]:
def main():
    # contoh file
    files = ["a.txt", "b.txt", "c.txt"]

    # bagi kerja ke 2 thread
    mid = len(files)//2
    t1 = threading.Thread(target=HitungKataDalamFile, args=(files[:mid],))
    t2 = threading.Thread(target=HitungKataDalamFile, args=(files[mid:],))

    # Start + join
    t1.start(); t2.start()
    t1.join(); t2.join()

    df = pd.DataFrame(sorted(jumlahKata.items(), key=lambda kv: kv[1], reverse=True));

    # Print hasil
    print(df.to_string())

<br>

# 5. Gunakan program

In [25]:
if __name__ == '__main__':
  main()

                   0    1
0                  a  300
1                and  260
2                 to  252
3                the  236
4                 we  172
5       segmentation  148
6                 of  128
7              model  124
8                 in  124
9                for  104
10            prompt  104
11                is   96
12              task   84
13              that   84
14              data   80
15               sam   72
16               our   68
17              with   68
18              this   68
19              mask   68
20             masks   64
21             image   60
22                on   60
23           dataset   56
24             tasks   56
25               can   52
26          training   52
27              shot   48
28                by   44
29                be   44
30             using   40
31              zero   40
32                 b   40
33        promptable   36
34        foundation   36
35            models   36
36               pre   36
37          